## Simulation of measured Fluorescence Polarization for a 1:1 binding assay

Plots measured fluorescence polarization for a range of protein:ligand K<sub>D</sub> as a function <br>
of protein concentration.<br> The concentration ([Fluoro]) of the fluorocently labeled ligand can be <br>
varied to see the impact on the impact on the binding curves.

### Run the following cell to activate the plot

In [12]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import ipywidgets as widgets

def FPbind(x, dmax, dmin, kd, pt):
    # FP based on Kd for a range of ligand concentrations at a fixed
    # fluorescently labeled protein concentration
    FP = dmin + ( (dmax - dmin) * (((kd + x + pt) - \
                                   (np.sqrt((kd + x + pt)**2 - \
                                            (4 * x * pt))))/(2*pt)))
    return FP


Fluoro = widgets.FloatSlider(
    value=2.5,
    min=0.001,
    max=10,
    step=0.01,
    description='[Fluor]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

Protein_Conc = widgets.FloatSlider(
    value=20,
    min=0.1,
    max=100,
    step=0.1,
    description='[Protein]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

Kd_range = widgets.FloatRangeSlider(
    value=[1, 200],
    min=0.001,
    max=300,
    step=0.01,
    description='Kd:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)


def response(change):
    global Kd_range, g, FPmax, FPmin, Fluoro
    i=0
    Conc_fluoro = Fluoro.value

    for Kd in np.linspace(Kd_range.value[0], Kd_range.value[1], 20):
        new_Y = FPbind(x_fit, FPmax, FPmin, Kd, Conc_fluoro)
        with g.batch_update():
            g.data[i]['y'] = new_Y
            g.data[i]['name'] = 'Kd = {:.2f}'.format(Kd)
        i+=1

min_ligand = 0.001
max_ligand = 10000

Conc_fluoro = Fluoro.value
FPmax = 140
FPmin = 30

fig = go.Figure()

x_fit = np.logspace(np.log10((min_ligand)*0.5), np.log10((max_ligand)*2), 500)

Kd_list = []
for Kd in np.linspace(Kd_range.value[0], Kd_range.value[1], 20):
    Kd_list.append(Kd)
    y_fit = FPbind(x_fit, FPmax, FPmin, Kd, Conc_fluoro)
    fig.add_trace(go.Scatter(x = x_fit,y = y_fit, name='Kd = {:.2f}'.format(Kd), showlegend = True, mode = "lines"))

fig.update_xaxes(type='log')
fig.update_xaxes(gridcolor='light gray',gridwidth = 0.2,showgrid = True,title_font_size = 16, mirror = True)
fig.update_yaxes(gridcolor='light gray',gridwidth = 0.2,showgrid = True,title_font_size = 16, mirror = True)
fig.update_layout(
                    template = 'simple_white',
                    height = 400,
                    width = 700,
                    xaxis = dict(title_text = "[Protein]"),
                    yaxis = dict(title_text = "Polarization"),
                    paper_bgcolor='aliceblue',
                    title = "Fluorescence Polarization Binding Assay")

g = go.FigureWidget(fig)

hbox = widgets.HBox([Fluoro, Kd_range])
display(g)
display(hbox)

Fluoro.observe(response, names="value")
Kd_range.observe(response, names="value")

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Kd = 1.00',
              'showlegend': True,
              'type': 'scatter',
              'uid': '7bce3071-28b3-4f10-ab29-ef38789b5c74',
              'x': array([5.00000000e-04, 5.17850731e-04, 5.36338759e-04, ..., 1.86449326e+04,
                          1.93105839e+04, 2.00000000e+04], shape=(500,)),
              'y': array([ 30.01571364,  30.01627462,  30.01685562, ..., 139.9940998 ,
                          139.9943032 , 139.99449959], shape=(500,))},
             {'mode': 'lines',
              'name': 'Kd = 11.47',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'afdd2910-a022-4d9e-88fb-120b2573f234',
              'x': array([5.00000000e-04, 5.17850731e-04, 5.36338759e-04, ..., 1.86449326e+04,
                          1.93105839e+04, 2.00000000e+04], shape=(500,)),
              'y': array([ 30.00393585,  30.00407637,  30.00422189, ..., 139.93234097,
             

## Simulation of measured Fluorescence Polarization for a competitive inhibition assay

Plots measured fluorescence polarization for a range of inhibitor K<sub>D</sub> as a function of inhibitor concentration.<br> 
The concentration ([Fluoro]) and binding constant (Kd Fluor) of the fluorocently labeled ligand can be varied to see the <br>
impact on the binding curves.

### Run the following cell to activate the plot

In [1]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import ipywidgets as widgets

def FPinhib(x, Yml1, Yl1, Kd1, L1t, Kd2, Mt):
    # returns fraction bound for FP inhibition assay
    # Yml1 = dmax, Yl1 = dmin, Kd1 = fixed binding constant, L1t = concProtein (labeled, fixed) 
    # Kd2 = Kd, Mt = protein concentration (fixed, unlabeled)
    L2t = x
    a = (Kd1 + Kd2 + L1t + L2t - Mt)
    b = Kd1*Kd2 + Kd1*(L2t-Mt) + Kd2*(L1t-Mt)
    c = -Mt*Kd1*Kd2
    
    theta = np.arccos( (-2 * a**3 + 9*a*b - 27*c) / (2*np.sqrt((a**2 - 3*b)**3)))
    
    d = ((2 * np.sqrt(a**2 - 3*b) * np.cos(theta/3)) - a)

    FP = Yl1 + ((Yml1-Yl1)*(d/(3*Kd1 + d))) 
    return FP


Fluoro = widgets.FloatSlider(
    value=2.5,
    min=0.001,
    max=10,
    step=0.01,
    description='[Fluor]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

Fluoro_Kd = widgets.FloatSlider(
    value=1.9,
    min=0.001,
    max=10,
    step=0.005,
    description='Kd Fluor:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)

Protein_Conc = widgets.FloatSlider(
    value=20,
    min=0.1,
    max=100,
    step=0.1,
    description='[Protein]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

Kd_range = widgets.FloatRangeSlider(
    value=[1, 200],
    min=0.001,
    max=300,
    step=0.01,
    description='Kd inhibitor:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)


def response(change):
    global Kd_range, g, FPmax, FPmin, Fluoro, Fluoro_Kd, Protein_Conc
    i=0
    Kd_fluoro = Fluoro_Kd.value
    Conc_fluoro = Fluoro.value
    Protein = Protein_Conc.value

    for Kd in np.linspace(Kd_range.value[0], Kd_range.value[1], 20):
        new_Y = FPinhib(x_fit, FPmax, FPmin, Kd_fluoro, Conc_fluoro, Kd, Protein)
        with g.batch_update():
            g.data[i]['y'] = new_Y
            g.data[i]['name'] = 'Kd = {:.2f}'.format(Kd)
        i+=1

min_ligand = 1
max_ligand = 5000

Kd_fluoro = Fluoro_Kd.value
Conc_fluoro = Fluoro.value
Protein = Protein_Conc.value
FPmax = 140
FPmin = 30

fig = go.Figure()

x_fit = np.logspace(np.log10((min_ligand)*0.5), np.log10((max_ligand)*2), 500)

Kd_list = []
for Kd in np.linspace(Kd_range.value[0], Kd_range.value[1], 20):
    Kd_list.append(Kd)
    y_fit = FPinhib(x_fit, FPmax, FPmin, Kd_fluoro, Conc_fluoro, Kd, Protein)
    fig.add_trace(go.Scatter(x = x_fit,y = y_fit, name='Kd = {:.2f}'.format(Kd), showlegend = True, mode = "lines"))

fig.update_xaxes(type='log')
fig.update_xaxes(gridcolor='light gray',gridwidth = 0.2,showgrid = True,title_font_size = 16, mirror = True)
fig.update_yaxes(gridcolor='light gray',gridwidth = 0.2,showgrid = True,title_font_size = 16, mirror = True)
fig.update_layout(
                    template = 'simple_white',
                    height = 400,
                    width = 700,
                    xaxis = dict(title_text = "[Inhibitor]"),
                    yaxis = dict(title_text = "Polarization"),
                    paper_bgcolor='aliceblue',
                    title = "Fluorescence Polarization Inhibition Assay")

g = go.FigureWidget(fig)

vbox1 = widgets.VBox([Fluoro, Protein_Conc])
vbox2 = widgets.VBox([Fluoro_Kd, Kd_range])
hbox = widgets.HBox([vbox1, vbox2])
display(g)
display(hbox)

Fluoro.observe(response, names="value")
Fluoro_Kd.observe(response, names="value")
Protein_Conc.observe(response, names="value")
Kd_range.observe(response, names="value")

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Kd = 1.00',
              'showlegend': True,
              'type': 'scatter',
              'uid': '924d1b9f-1ea1-406a-a526-c86f95913dee',
              'x': array([5.00000000e-01, 5.10022461e-01, 5.20245822e-01, ..., 9.61084123e+03,
                          9.80348980e+03, 1.00000000e+04], shape=(500,)),
              'y': array([129.10043493, 129.0951238 , 129.08970128, ...,  30.12056776,
                           30.11819674,  30.11587237], shape=(500,))},
             {'mode': 'lines',
              'name': 'Kd = 11.47',
              'showlegend': True,
              'type': 'scatter',
              'uid': '0687e12c-a82d-45e0-bc1d-9705191bf0c2',
              'x': array([5.00000000e-01, 5.10022461e-01, 5.20245822e-01, ..., 9.61084123e+03,
                          9.80348980e+03, 1.00000000e+04], shape=(500,)),
              'y': array([129.19558303, 129.19227039, 129.18889021, ...,  31.36426813,
             